In [ ]:
!pip install GitPython
!pip install sagemaker
!pip install s3fs
!pip install aws-mlops
#!pip uninstall -y aws-mlops

In [ ]:
import os
import sys
#os.environ['KEY'] = 'aws-mlops/test/ee317d661bb0f13f5eda338d07221ed08496e186/2022-01-24-19-07-56'
import config as config
import definitions as definitions
#sys.path.insert(1, os.getcwd() + '/..')
from aws_mlops.mlops import MLOps
from aws_mlops.data_storage import DataStorage
print(config.environment, config.key)

In [ ]:
# initialization
mo = MLOps(config)
ds = DataStorage(config.source_bucket, config.test_data_key)

In [ ]:
# create definition files
mo.create_step_functions_definition_files(definitions)

In [ ]:
# load data for pretraining
df = ds.local_read('/home/jovyan/example/processing/raw_data', config.raw_data_filename)
ds.save_on_s3(df, config.raw_data_filename, f's3://{config.source_bucket}/{config.repo_name}/{config.branch}/raw_data', header = True, index = False)

In [ ]:
# run only processing of pretraining
mo.run_state_machine('pretraining')

In [ ]:
# run only modeling and testing prediction with test data
mo.run_state_machine('modeling')

In [ ]:
# load data for preinference
df = ds.local_read('/home/jovyan/example/processing/new_data', config.new_data_filename)
ds.save_on_s3(df, config.new_data_filename, f's3://{config.source_bucket}/{config.repo_name}/{config.branch}/new_data', header = True, index = False)

In [ ]:
# run only prediction with new test data
mo.run_state_machine('prediction')

In [ ]:
# run all cycle: pretraining, modeling and testing prediction with test data
mo.run_state_machine('mlops')